In [667]:
import math

In [668]:
class LCG:
    def __init__(self, seed, a=561860773102413563, c=0, m=2**60-93):
        self.seed = seed
        self.a = a
        self.c = c
        self.m = m
        self.state = seed

    def next(self):
        self.state = (self.a * self.state + self.c) % self.m
        return self.state / self.m  # Normalize to [0, 1)
    
    def next_in_range(self, a, b):
        return a + (b - a) * self.next()

In [669]:
class MonteCarlo:
    def __init__(self, N, PRNG_object):
        self.N = N
        self.PRNG = PRNG_object
    
    def integrate(self, f, a, b):
        mult = (b - a) / self.N
        
        generatedValues = []
        for _ in range(self.N):
            randomArg = self.PRNG.next_in_range(a, b)
            randomFuncVal = f(randomArg)

            generatedValues.append(randomFuncVal)
        
        return mult * sum(generatedValues)

In [670]:
class CDM:
    def __init__(self, h):
        self.h = h
    
    def diff(self, f, x):
        numerator = f(x + self.h) - f(x - self.h)
        denominator = 2 * self.h

        return numerator / denominator

In [671]:
class Newton:
    def __init__(self, f, CDM_object, tol=1e-6, max_iter=1000):
        self.f = f
        self.CDM = CDM_object
        self.tol = tol 
        self.max_iter = max_iter

    def solve(self, y, x0):
        x = x0
        for _ in range(self.max_iter):
            f_x = self.f(x) - y
            f_prime_x = self.CDM.diff(self.f, x)
            if abs(f_prime_x) < 1e-10:
                raise ValueError("Derivative is zero, method fails.")
            x_new = x - f_x / f_prime_x
            if abs(x_new - x) < self.tol:
                return x_new
            x = x_new
        # return x_new
        raise ValueError(f"Method did not converge.({x_new})")

In [672]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(100000, lcg)
    
    def pdf(x): # f_X
        return 1 / (math.sqrt(0.4 * math.pi) * x) \
            * math.exp(-(math.log(x) - 2)**2 / 0.4)

    def cdf(x): # F_X
        return monteCarlo.integrate(pdf, 0, x)

    cdm = CDM(h=1e-6)
    newton = Newton(cdf, cdm, 0.001, 100)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

In [673]:
def foo(x):
    return 1 - (50/x)**(25)

def inv_foo(y):
    return 50 / ((1 - y)**(1/25))

In [674]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(100000, lcg)
    
    def pdf(x): # f_X
        return (1 / 2) * (50 / x)**(26)

    def cdf(x): # F_X
        return monteCarlo.integrate(pdf, 50, x)

    for i in range(51, 60):
        print(f'{foo(i)}   {cdf(i)}')

    cdm    = CDM(h=1e-6)
    newton = Newton(cdf, cdm, tol=1e-6, max_iter=1000)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    print(f'-'*200)

    print(inv_foo(0.355))
    print(inverse(0.355, 50))

0.3904691294717214   0.3903659380649333
0.6248831977460355   0.6238114760128302
0.7670013694961044   0.7671979098566637
0.8539820950870863   0.8548339549781164
0.9077040018229359   0.9078868064569301
0.9411766934477461   0.93676785737656
0.962209841226469   0.9604220950801329
0.9755347155613031   0.97645330210309
0.9840430524567241   0.9834782007801411
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
50.884746556271914


ValueError: Method did not converge.(51.328430015992)

In [ ]:
def foo(x):
    # return math.sin(x)
    return x**3 - 2 * x - 5

def inv_foo(y):
    return math.asin(y)

In [ ]:
if __name__ == '__main__':
    lcg        = LCG(seed=42)
    monteCarlo = MonteCarlo(100000, lcg)
    
    def pdf(x): # f_X
        # return 1 / (math.sqrt(0.4 * math.pi) * x) \
        #     * math.exp(-(math.log(x) - 2)**2 / 0.4)
        return (1 / 2) * (50 / x)**(26)

    def cdf(x): # F_X
        # return monteCarlo.integrate(pdf, 0, x)
        return monteCarlo.integrate(pdf, 50, x)

    # for i in range(51, 60):
    #     print(f'{foo(i)}   {cdf(i)}')

    cdm    = CDM(h=1e-6)
    newton = Newton(foo, cdm, 0.0001, 100)

    def inverse(y, x0): # x = f^-1(y)
        return newton.solve(y, x0)

    print(f'foo: {foo(3.1519)}')
    # print(f'cdf(51): {cdf(51)}')

    # print(f'')

    # print(inv_foo(0.625))
    print(newton.solve(20, 0))

foo: 20.008667371359
3.1515882293555033
